In [1]:
!python --version

Python 3.9.7


In [2]:
#신경망구축에 필요한 tensorflow
#라이브러리와 numpy 그리고 시각화도구
#matplotlib를 import 한다.
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#Variable의 랜덤함수 시드를 설정한다.
tf.random.set_seed(777) #변경한 버전에서는 랜덤시드 사용 불가능해서 주석처리함.

In [3]:
print(tf.__version__)

2.8.0


In [4]:
#MinMaxScaler 정의 -> data를 0부터 1사이의 값으로 변환(normalize)
def MinMaxScaler(data):
    numerator = data - np.min(data, 0)
    denominator = np.max(data, 0) - np.min(data, 0)
    return numerator / (denominator + 1e-7)

In [5]:
#train Parameter 상수값 설정
seq_length = 7 #7일 단위로 학습시키고 8일째를 예측
input_dim = 14 #input 데이터의 개수
hidden_dim = 16 #은닉층의 개수
output_dim = 1 # target label의 개수(사과가격)
learning_rate = 0.01
iterations = 10001

In [6]:
#값 설정이 끝나면 위에서 만든 데이터 셋을 불러온다. 그 후 데이터양에 따라 적절하게 Training Data와 Test Data로 분할한다.
#data load
#만들어둔 데이터셋을 Load한다.
xy = np.loadtxt('./apple_data.csv', delimiter=',')
xy = MinMaxScaler(xy) #Normalize
x = xy #전체 데이터(input)
y = xy[:,[-1]] #target Label(사과가격)

In [7]:
#build dataset
#Normalize가 끝난 데이터를 7일단위로 슬라이스 해서 3차원형태로 만든다.
dataX = []
dataY = []
for i in range(0, len(y) - seq_length):
    _x = x[i: i+seq_length]
    _y = y[i+seq_length]
    dataX.append(_x)
    dataY.append(_y)
#train/test set 나누기
train_size = int(len(dataY) * 0.7) #train size = 70%
test_size = len(dataY) - train_size #test size = 30%
trainX, testX = np.array(dataX[0:train_size]), np.array(dataX[train_size:len(dataX)])
trainY, testY = np.array(dataY[0:train_size]), np.array(dataY[train_size:len(dataY)])

In [16]:
trainX

array([[[0.21750663, 0.27918782, 0.        , ..., 0.22323462,
         0.11764706, 0.22404372],
        [0.25994695, 0.33756345, 0.        , ..., 0.25056948,
         0.11764706, 0.27595628],
        [0.29442971, 0.30203046, 0.        , ..., 0.25056948,
         0.11764706, 0.27322404],
        ...,
        [0.22811671, 0.21827411, 0.        , ..., 0.24829157,
         0.11764706, 0.22404372],
        [0.28912467, 0.23857868, 0.        , ..., 0.25056948,
         0.11764706, 0.26502732],
        [0.19893899, 0.23096447, 0.        , ..., 0.20728929,
         0.11764706, 0.22131147]],

       [[0.25994695, 0.33756345, 0.        , ..., 0.25056948,
         0.11764706, 0.27595628],
        [0.29442971, 0.30203046, 0.        , ..., 0.25056948,
         0.11764706, 0.27322404],
        [0.22015915, 0.30964467, 0.        , ..., 0.20045558,
         0.11764706, 0.23770492],
        ...,
        [0.28912467, 0.23857868, 0.        , ..., 0.25056948,
         0.11764706, 0.26502732],
        [0.1

In [8]:
trainX.shape

(249, 7, 17)

In [9]:
testX.shape

(108, 7, 17)

In [10]:
m1,n1,r1 = trainX.shape
out_trainX = np.column_stack((np.repeat(np.arange(m1),n1),trainX.reshape(m1*n1,-1)))
out_df_trainX = pd.DataFrame(out_trainX)

In [17]:
out_df_trainX.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,0.0,0.217507,0.279188,0.0,0.183099,0.148148,0.102041,0.427660,0.406977,0.698413,0.020408,0.451852,0.34,0.000000,0.134571,0.223235,0.117647,0.224044
1,0.0,0.259947,0.337563,0.0,0.253521,0.283951,0.183673,0.468085,0.453488,0.699735,0.020408,0.577778,0.24,0.289157,0.164733,0.250569,0.117647,0.275956
2,0.0,0.294430,0.302030,0.0,0.316901,0.370370,0.306122,0.351064,0.186047,0.387566,0.020408,0.651852,0.00,0.000000,0.160093,0.250569,0.117647,0.273224
3,0.0,0.220159,0.309645,0.0,0.183099,0.160494,0.122449,0.406383,0.325581,0.601852,0.020408,0.444444,0.21,0.120482,0.136891,0.200456,0.117647,0.237705
4,0.0,0.228117,0.218274,0.0,0.366197,0.382716,0.306122,0.461702,0.720930,0.787037,0.020408,0.000000,0.61,0.614458,0.141531,0.248292,0.117647,0.224044
5,0.0,0.289125,0.238579,0.0,0.176056,0.209877,0.163265,0.457447,0.500000,0.679894,0.020408,0.022222,0.51,0.590361,0.183295,0.250569,0.117647,0.265027
6,0.0,0.198939,0.230964,0.0,0.042254,0.074074,0.000000,0.438298,0.534884,0.732804,0.040816,0.222222,0.61,0.373494,0.143852,0.207289,0.117647,0.221311
7,1.0,0.259947,0.337563,0.0,0.253521,0.283951,0.183673,0.468085,0.453488,0.699735,0.020408,0.577778,0.24,0.289157,0.164733,0.250569,0.117647,0.275956
8,1.0,0.294430,0.302030,0.0,0.316901,0.370370,0.306122,0.351064,0.186047,0.387566,0.020408,0.651852,0.00,0.000000,0.160093,0.250569,0.117647,0.273224
9,1.0,0.220159,0.309645,0.0,0.183099,0.160494,0.122449,0.406383,0.325581,0.601852,0.020408,0.444444,0.21,0.120482,0.136891,0.200456,0.117647,0.237705


In [14]:
m2,n2,r2 = testX.shape
out_testX = np.column_stack((np.repeat(np.arange(m2),n2),testX.reshape(m2*n2,-1)))
out_df_testX = pd.DataFrame(out_testX)

In [15]:
out_df_testX

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,0.0,0.742706,0.781726,0.000000,0.225352,0.185185,0.142857,0.825532,0.418605,0.662698,0.612245,0.622222,0.16,0.108434,0.703016,0.744875,0.509804,0.775956
1,0.0,0.745358,0.796954,0.000000,0.190141,0.111111,0.102041,0.804255,0.325581,0.628307,0.612245,0.592593,0.39,0.060241,0.716937,0.731207,0.460784,0.775956
2,0.0,0.748011,0.763959,0.000000,0.394366,0.358025,0.306122,0.840426,0.430233,0.724868,0.591837,0.274074,0.78,0.674699,0.712297,0.747153,0.460784,0.762295
3,0.0,0.785146,0.601523,0.546154,0.246479,0.222222,0.285714,0.876596,0.848837,0.958995,0.591837,0.000000,0.93,0.650602,0.598608,0.840547,0.411765,0.702186
4,0.0,0.787798,0.786802,0.000000,0.309859,0.308642,0.285714,0.800000,0.279070,0.560847,0.571429,0.837037,0.24,0.289157,0.749420,0.797267,0.313725,0.797814
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
751,107.0,0.185676,0.167513,0.000000,0.880282,0.765432,0.714286,0.202128,0.116279,0.255291,0.000000,0.511111,0.36,0.433735,0.088167,0.246014,0.215686,0.161202
752,107.0,0.114058,0.063452,0.000000,0.584507,0.629630,0.816327,0.017021,0.069767,0.058201,0.020408,0.674074,0.00,0.000000,0.032483,0.223235,0.313725,0.068306
753,107.0,0.175066,0.144670,0.000000,0.373239,0.358025,0.469388,0.185106,0.116279,0.226190,0.000000,0.370370,0.44,0.120482,0.064965,0.220957,0.313725,0.144809
754,107.0,0.286472,0.256345,0.000000,0.485915,0.469136,0.346939,0.404255,0.441860,0.503968,0.020408,0.451852,0.60,0.722892,0.150812,0.312073,0.352941,0.267760


In [20]:
# 데이터 셋 분할이 끝났으면 해당 데이터를 받을 Placeholder와 인공신경망 구조인 LSTM을 구축한다.
#input placeholders
#input placeholder의 파라미터로 seq_length,input_dim를 넘겨준다.
#X = tf.placeholder(tf.float32, [None, seq_length, input_dim]) #3차원 형태의 input placeholder
#X = tf.Tensor([None seq_length input_dim], shape=(3,), dtype=float32)
X = tf.constant(trainX, dtype=float, shape=(3,))
Y = tf.constant(trainY, dtype=float, shape=(3,))
#Y = tf.placeholder(tf.float32, [None, 1]) #2차원 형태의 output placeholder
#Y = tf.Tensor([None 1], shape=(2,), dtype=float32)

TypeError: Eager execution of tf.constant with unsupported shape. Tensor [[[0.21750663 0.27918783 0.         ... 0.22323462 0.11764706 0.22404371]
  [0.25994694 0.33756346 0.         ... 0.25056946 0.11764706 0.27595627]
  [0.29442972 0.30203044 0.         ... 0.25056946 0.11764706 0.27322406]
  ...
  [0.2281167  0.21827412 0.         ... 0.24829157 0.11764706 0.22404371]
  [0.28912467 0.23857868 0.         ... 0.25056946 0.11764706 0.2650273 ]
  [0.198939   0.23096447 0.         ... 0.2072893  0.11764706 0.22131148]]

 [[0.25994694 0.33756346 0.         ... 0.25056946 0.11764706 0.27595627]
  [0.29442972 0.30203044 0.         ... 0.25056946 0.11764706 0.27322406]
  [0.22015914 0.30964467 0.         ... 0.20045558 0.11764706 0.23770492]
  ...
  [0.28912467 0.23857868 0.         ... 0.25056946 0.11764706 0.2650273 ]
  [0.198939   0.23096447 0.         ... 0.2072893  0.11764706 0.22131148]
  [0.38726792 0.35532996 0.         ... 0.40318906 0.11764706 0.33333334]]

 [[0.29442972 0.3020304

In [23]:
#build a LSTM network (build Rnn)
#tensorflow를 이용하면 LSTM 구조를 cell 형태로 빠르고 쉽게 생성해준다.
#cell = tf.keras.layers.LSTMCell(num_units=hidden_dim, state_is_tuple=True, activation=tf.tanh)
cell = tf.keras.layers.LSTMCell(
    units,
    activation='tanh',
    recurrent_activation='sigmoid',
    use_bias=True,
    kernel_initializer='glorot_uniform',
    recurrent_initializer='orthogonal',
    bias_initializer='zeros',
    unit_forget_bias=True,
    kernel_regularizer=None,
    recurrent_regularizer=None,
    bias_regularizer=None,
    kernel_constraint=None,
    recurrent_constraint=None,
    bias_constraint=None,
    dropout=0.0,
    recurrent_dropout=0.0,
    **kwargs
)
#생성된 cell을 dynamic_rnn함수로 활성화 시켜준다. 파라미터로 생성된 cell과 input placeholder를 넘겨주고 리턴값으로 output을 받는다.
outputs, _states = tf.nn.dynamic_rnn(cell, X, dtype = tf.float32)
#그 후 출력된 output을 fully_connected에 통과 시켜 최종 예측값을 리턴 받는다.
#Y_pred = tf.contrib.layers.fully_connected(outputs[:, -1], output_dim, activation_fn=tf.tanh)
#Y_pred = tf.layers.fully_connected(outputs[:, -1], output_dim, activation_fn=tf.tanh)
Y_pred = tf.keras.layers.fully_connected(outputs[:, -1], output_dim, activation_fn=tf.tanh)

NameError: name 'units' is not defined

In [21]:
print (type(Y_pred))

NameError: name 'Y_pred' is not defined